In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
postulantes_ge = pd.read_csv("../../Data/fiuba_entrenamiento/postulantes_ge.csv")
postulantes_ge.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [4]:
postulantes_educacion = pd.read_csv("../../Data/fiuba_entrenamiento/postulantes_educacion.csv")
postulantes_educacion.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [5]:
postulantes_ge['fechanacimiento'] = pd.to_datetime(postulantes_ge['fechanacimiento'], errors = 'coerce')
postulantes_ge['edad'] = datetime.now() - postulantes_ge['fechanacimiento']
postulantes_ge['edad']=pd.TimedeltaIndex(postulantes_ge['edad']).days;
postulantes_ge['edad']=np.floor(postulantes_ge['edad']/365);
postulantes_ge.drop(['fechanacimiento'],axis=1, inplace=True);
postulantes_ge.head()

,idpostulante,sexo,edad
0,NM5M,FEM,47.0
1,5awk,FEM,55.0
2,ZaO5,FEM,39.0
3,NdJl,MASC,49.0
4,eo2p,MASC,37.0


In [6]:
def mayor_estado(estados):
    retval="Abandonado"
    for elem in estados:
        if(elem=="En Curso"): retval=elem;
        if(elem=="Graduado"): retval=elem;
    return retval
group = postulantes_educacion.groupby(['idpostulante','nombre'], as_index=False).agg({'estado':mayor_estado})
group.head()

,idpostulante,nombre,estado
0,0z5Dmrd,Universitario,Graduado
1,0z5JW1r,Otro,Graduado
2,0z5JW1r,Secundario,Graduado
3,0z5JW1r,Terciario/Técnico,Graduado
4,0z5JW1r,Universitario,Abandonado


In [7]:
group2=group.pivot_table(values='estado', index=['idpostulante'], columns='nombre', aggfunc='first').reset_index()
group2=group2.fillna("-")
group2.head()

nombre,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,-,-,-,-,-,-,Graduado
1,0z5JW1r,-,-,Graduado,-,Graduado,Graduado,Abandonado
2,0z5VvGv,-,-,-,-,Graduado,-,-
3,0zB01pE,-,-,-,-,-,Graduado,Abandonado
4,0zB026d,-,-,-,-,Graduado,Graduado,-


In [8]:
postulantes=pd.merge(postulantes_ge,group2, on='idpostulante',how='left');
postulantes.fillna("-")
postulantes.head()

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,NM5M,FEM,47.0,-,-,-,-,Graduado,-,-
1,5awk,FEM,55.0,-,-,-,-,-,Graduado,Graduado
2,ZaO5,FEM,39.0,-,-,-,-,-,Graduado,Abandonado
3,NdJl,MASC,49.0,-,-,-,En Curso,-,-,Graduado
4,eo2p,MASC,37.0,-,Graduado,-,-,Graduado,-,Graduado


In [9]:
postulantes.edad=postulantes.edad.fillna(200)
postulantes=postulantes.fillna("-")
postulantes=postulantes.replace("NaN", "-")
postulantes.sample(20)

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
339044,vV9mNkx,MASC,24.0,-,-,-,-,Graduado,-,En Curso
111164,pzjRDwW,FEM,48.0,-,-,-,-,Graduado,-,-
143524,MV6QbBx,FEM,31.0,-,-,-,Graduado,-,-,Graduado
333133,6r5348r,MASC,22.0,-,-,-,-,Graduado,-,-
35555,YBMDwP,FEM,33.0,-,Graduado,Graduado,-,-,-,Graduado
485194,W9WBW3B,MASC,20.0,-,-,-,-,-,-,En Curso
142688,6r5Nk4r,MASC,22.0,-,-,-,-,Graduado,-,-
402932,RzroRox,FEM,31.0,-,-,-,-,-,Graduado,-
330726,96zexrZ,MASC,41.0,-,-,-,-,Graduado,-,-
341223,1QPr4b3,FEM,21.0,-,-,-,-,Graduado,-,En Curso


In [10]:
def completar_estudio_menor(estudio_menor,estudio_mayor):
    if(estudio_mayor=="Graduado"):
        return "Graduado"
    return estudio_menor;
completar_estudio_menor=np.vectorize(completar_estudio_menor)
postulantes['Secundario']=completar_estudio_menor(postulantes['Secundario'],postulantes['Universitario'])
postulantes.head()

,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,NM5M,FEM,47.0,-,-,-,-,Graduado,-,-
1,5awk,FEM,55.0,-,-,-,-,Graduado,Graduado,Graduado
2,ZaO5,FEM,39.0,-,-,-,-,-,Graduado,Abandonado
3,NdJl,MASC,49.0,-,-,-,En Curso,Graduado,-,Graduado
4,eo2p,MASC,37.0,-,Graduado,-,-,Graduado,-,Graduado


In [11]:
print(postulantes.shape)

(505382, 10)


In [12]:
postulantes.drop_duplicates(['idpostulante'],keep='first',inplace=True)
postulantes.to_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv",index=False)